This notebook demonstrates the method to produce the most basic simulation under consideration using the package `varsim`. The inputs here are:
   - a set of pointings (for simplicity, read in from a csv file)
   - a population of astrophysical sources described by a subclass of `varsim.BasePopulation`. This class must implement the abstract methods of `varsim.BasePopulation`, and provide an index for each source and the model parameters
   - a model for the sources described by a subclass of `varsim.BaseModel`. Again this subclass must implement all of the abstract methods and properties of `varsim.BaseModel`. The essential functionality of this class is to represent an astrophysical source as a model with model parameters, given which, this class has methods of predicting the model flux as a function of time at the top of the earth's atmosphere (ie. no sky noise included).
   - The simulation will be performed using the class `varsim.BasicSimulation` which is a subclass of the abstract base class `varsim.BaseSimulation` and implements concrete methods and properties necessary for the simulation. These methods and properties only use the abstract properties and methods of `BaseModel` and `BasePopulation`, and are therefore guaranteed to with any subclass.

##  Imports

In [1]:
import os

In [2]:
from opsimsummary import HealpixTiles, OpSimOutput

In [3]:
import numpy as np
import pandas as pd

In [5]:
from lsst.sims.photUtils import BandpassDict

In [6]:
from varsim import BasePopulation, BasicSimulation, BaseModel

In [7]:
#hptiles = HealpixTiles(nside=256, 
#                       preComputedMap='/Users/rbiswas/data/LSST/OpSimData/healpixelized_MINION_1016_256_64_indexed.db')

In [8]:
from lsst.sims.catUtils.supernovae import SNObject

In [9]:
import analyzeSN as ans

In [10]:
import varsim

## The quantities of interest

####  Pointings

In [11]:
# The set of pointings randomly kept for convenience.
example_data = varsim.example_data
pointings_File = os.path.join(example_data, 'example_pointings.csv')
pointings = pd.read_csv(pointings_File, index_col='obsHistID')

In [12]:
# To show what this looks like
print(len(pointings))
pointings.head()

14


,sessionID,propID,fieldID,fieldRA,fieldDec,filter,expDate,expMJD,night,visitTime,...,moonBright,darkBright,rawSeeing,wind,humidity,slewDist,slewTime,fiveSigmaDepth,ditheredRA,ditheredDec
obsHistID,,,,,,,,,,,,,,,,,,,,,
1,1016,54,316,1.676483,-1.082473,y,2922,59580.033829,0,34.0,...,0.0,105.610378,0.920473,0.0,0.0,1.620307,0.000000,21.021236,1.643930,-1.108924
14,1016,54,1118,1.709686,-0.607725,y,3424,59580.039637,0,34.0,...,0.0,103.104582,0.903960,0.0,0.0,0.055856,4.599568,21.067249,1.691083,-0.634176
13,1016,54,1220,1.664386,-0.566519,y,3386,59580.039190,0,34.0,...,0.0,101.147431,0.903960,0.0,0.0,0.048200,4.776511,21.091149,1.646286,-0.592970
12,1016,54,1212,1.608166,-0.575819,y,3347,59580.038741,0,34.0,...,0.0,98.109397,0.903960,0.0,0.0,0.055665,4.739998,21.128319,1.589958,-0.602270
11,1016,54,1110,1.651280,-0.618574,y,3308,59580.038293,0,34.0,...,0.0,100.125895,0.903960,0.0,0.0,0.056173,4.499824,21.103635,1.632535,-0.645025


In [13]:
# We did not need to have all these columns. The essential columns are ra, dec, filter, 
# fivesigmadepth. It is also good to have fieldID, etc.

### Population of astrophysical sources. 
Here I use supernovae with the SALT model.  I keep two supernovae in a completely random way without attempting to make sense. The important parts are  that no matter how the Population model is implemented, it has:
- `modelparams` : the method which takes the unique index of a supernova in the population and provides its model parameters as a dictionary. An important requirement is that the dictionary as the keys `ra`, `dec`, while the other parameters are completely user dependent
- `idxvalues` : a property which is a sequence of indices. Here the sequence is implemented as a tuple, which is perhaps how it should be for large simulations.

In [14]:
class SALTPopulation(BasePopulation):
    def __init__(self):
        self.x0 = [5.0e-2, 3.e-5]
        self.x1 = [0, .1]
        self.c = [-0.2, 0.5]
        self.t0 = [59581., 59580. ]
        self.z = [0.5, 0.6]
        self.ra = [30., 30.]
        self.dec = [-45., -45.]
        
    def modelparams(self, idx):
        return dict(x0=self.x0[idx], x1=self.x1[idx], c=self.c[idx], t0=self.t0[idx],
                    z=self.z[idx], ra=self.ra[idx], dec=self.dec[idx])
    
    @property
    def idxvalues(self):
        return (x for x in (0, 1))
    @property
    def hasPositions(self):
        return True
    @property
    def positions(self):
        return (0, 1)

#### Instantiate the population and demonstrate

In [15]:
sp = SALTPopulation()

In [16]:
# The indices
idxs = tuple(sp.idxvalues)
print(idxs)

(0, 1)


In [17]:
# The model parameters
sp.modelparams(1)

{'c': 0.5,
 'dec': -45.0,
 'ra': 30.0,
 't0': 59580.0,
 'x0': 3e-05,
 'x1': 0.1,
 'z': 0.6}

and the other one

In [18]:
sp.modelparams(0)

{'c': -0.2,
 'dec': -45.0,
 'ra': 30.0,
 't0': 59581.0,
 'x0': 0.05,
 'x1': 0,
 'z': 0.5}

### Model of source
- Model : Subclass of `varsim.BaseModel` with its methods implemented. Here I use `lsst.sims.catUtils.supernovae.SNObject` functionality to provide  functions
- Provides `minMjd` and `maxMjd` to restrict the set of pointings over which we will build the light curve.
- `modelFlux`: concrete implementation of abstract method `BaseModel.modelFlux` providing the flux, given the model parameters and `mjd`, `bandpass`.

In [19]:
# We will need the LSST bandpasses. Let us load them using the catsim method
bandpassdict = BandpassDict.loadTotalBandpassesFromFiles()

In [20]:
class Model(SNObject, BaseModel):
    def setModelParameters(self, **params):
        paramDict = params.copy()
        self.setCoords(ra=params['ra'], dec=params['dec'])
        for key in ('ra', 'dec'):
            params.pop(key)
        self.set(**params)
        self.mwEBVfromMaps()
    
    @property
    def minMjd(self):
        return self.mintime()
    @property
    def maxMjd(self):
        return self.maxtime()
    
    def modelFlux(self, mjd, bandpassobj):
        return self.catsimBandFlux(bandpassobject=bandpassobj,
                                      time=mjd)



#### Instantiate the model and demonstrate the functionality

In [21]:
model = Model()

In [22]:
# Let us set parameters (using the method `setModelParameters` in the abstract class
# to the parameters of the first object in SALTParameters)
model.setModelParameters(**sp.modelparams(0))    

In [23]:
# Now, this should predict fluxes (in maggies) given a time, and a bandpassobject
f = (model.modelFlux(mjd=59580, bandpassobj=bandpassdict['y']), model.modelFlux(mjd=59580, bandpassobj=bandpassdict['g']))
print(f, -2.5 * np.log10(f))

((2.6960816540605439e-06, 2.5625595265137595e-06), array([ 13.9231674 ,  13.97831509]))


## Simulation
Now we are ready to do the simulation using the class `BasicSimulation`

In [24]:
bsim = BasicSimulation(sp, model, pointings=pointings, rng=np.random.RandomState(0), 
                       maxObsHistID=1000000, pointingColumnDict=None,
                       pruneWithRadius=False)

In [38]:
!rm *.hdf

In [39]:
bsim.write_simulation(phot_output='sim_phot_all.hdf', pop_output='sim_pop.hdf', method='hdf', key='sim1')

writing 0, False, sim1
writing 1, False, sim1


In [40]:
phot_sim_df = pd.read_hdf('sim_phot_all.hdf', key='sim1')

In [41]:
phot_sim_df

,obsHistID,objid,expMJD,filter,ModelFlux,fieldID,flux,fluxerr,deviations,zp,zpsys
pid,,,,,,,,,,,
1000000,1,0,59580.033829,y,2.696882e-06,316,2.694083e-06,1.640065e-09,-1.706270,0.0,ab
14000000,14,0,59580.039637,y,2.697018e-06,1118,2.700188e-06,1.624962e-09,1.950775,0.0,ab
13000000,13,0,59580.039190,y,2.697008e-06,1220,2.696183e-06,1.617539e-09,-0.509652,0.0,ab
12000000,12,0,59580.038741,y,2.696997e-06,1212,2.696293e-06,1.606565e-09,-0.438074,0.0,ab
11000000,11,0,59580.038293,y,2.696986e-06,1110,2.694965e-06,1.613773e-09,-1.252795,0.0,ab
10000000,10,0,59580.037847,y,2.696976e-06,1021,2.698242e-06,1.628304e-09,0.777490,0.0,ab
9000000,9,0,59580.037401,y,2.696965e-06,957,2.694317e-06,1.641323e-09,-1.613898,0.0,ab
8000000,8,0,59580.036955,y,2.696955e-06,857,2.696606e-06,1.639033e-09,-0.212740,0.0,ab
7000000,7,0,59580.036509,y,2.696945e-06,770,2.695478e-06,1.637419e-09,-0.895467,0.0,ab


In [42]:
pop_sim_df = pd.read_hdf('sim_pop.hdf', key='sim1')

In [43]:
pop_sim_df

,c,dec,ra,t0,x0,x1,z
0,-0.2,-45.0,30.0,59581.0,0.05000,0.0,0.5
1,0.5,-45.0,30.0,59580.0,0.00003,0.1,0.6


# Scratch

In [38]:
bsim.lc(1).index.values.size

14

In [24]:
np.unique(bsim.lc(0).index.values).size

14

In [25]:
bsim.lc(0).columns

Index([u'objid', u'expMJD', u'filter', u'ModelFlux', u'fieldID', u'flux',
       u'fluxerr', u'deviations', u'zp', u'zpsys'],
      dtype='object')

In [26]:
bsim.write_lc(0, 'test_0.hdf', 'hdf')

0 a


/usr/local/miniconda/lib/python2.7/site-packages/tables/path.py:100: NaturalNameWarning: object name is not a valid Python identifier: '0'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  NaturalNameWarning)


In [27]:
lc_0 = pd.read_hdf('test_0.hdf')

In [28]:
lc_0

,objid,expMJD,filter,ModelFlux,fieldID,flux,fluxerr,deviations,zp,zpsys
pid,,,,,,,,,,
1000000,0,59580.033829,y,0.000003,316,0.000003,1.640065e-09,-1.706270,0.0,ab
14000000,0,59580.039637,y,0.000003,1118,0.000003,1.624962e-09,1.950775,0.0,ab
13000000,0,59580.039190,y,0.000003,1220,0.000003,1.617539e-09,-0.509652,0.0,ab
12000000,0,59580.038741,y,0.000003,1212,0.000003,1.606565e-09,-0.438074,0.0,ab
11000000,0,59580.038293,y,0.000003,1110,0.000003,1.613773e-09,-1.252795,0.0,ab
10000000,0,59580.037847,y,0.000003,1021,0.000003,1.628304e-09,0.777490,0.0,ab
9000000,0,59580.037401,y,0.000003,957,0.000003,1.641323e-09,-1.613898,0.0,ab
8000000,0,59580.036955,y,0.000003,857,0.000003,1.639033e-09,-0.212740,0.0,ab
7000000,0,59580.036509,y,0.000003,770,0.000003,1.637419e-09,-0.895467,0.0,ab


In [22]:
bsim.write_simulation('test_sim.hdf', 'hdf', key='test', clobber=False)

writing 0, False, test
writing 1, False, test


In [23]:
simdf = pd.read_hdf('test_sim.hdf')

In [24]:
simdf

,objid,expMJD,filter,ModelFlux,fieldID,flux,fluxerr,deviations,zp,zpsys
pid,,,,,,,,,,
1000000,0,59580.033829,y,2.696882e-06,316,2.696991e-06,1.640065e-09,0.066517,0.0,ab
14000000,0,59580.039637,y,2.697018e-06,1118,2.697510e-06,1.624962e-09,0.302472,0.0,ab
13000000,0,59580.039190,y,2.697008e-06,1220,2.695981e-06,1.617539e-09,-0.634322,0.0,ab
12000000,0,59580.038741,y,2.696997e-06,1212,2.696414e-06,1.606565e-09,-0.362741,0.0,ab
11000000,0,59580.038293,y,2.696986e-06,1110,2.695901e-06,1.613773e-09,-0.672460,0.0,ab
10000000,0,59580.037847,y,2.696976e-06,1021,2.696391e-06,1.628304e-09,-0.359553,0.0,ab
9000000,0,59580.037401,y,2.696965e-06,957,2.695631e-06,1.641323e-09,-0.813146,0.0,ab
8000000,0,59580.036955,y,2.696955e-06,857,2.694126e-06,1.639033e-09,-1.726283,0.0,ab
7000000,0,59580.036509,y,2.696945e-06,770,2.697235e-06,1.637419e-09,0.177426,0.0,ab


In [25]:
bsim.write_population()

TypeError: write_population() takes at least 3 arguments (1 given)

In [23]:
df = pd.read_hdf('test.hdf', key='test')

In [36]:
bsim.lc(1).to_hdf('test.hdf', key='test', mode='w', append=True, format='t')

In [37]:
bsim.lc(0).to_hdf('test.hdf', key='test', mode='a', append=True, format='t')

In [38]:
df = pd.read_hdf('test.hdf', key='test')

In [39]:
df

,objid,expMJD,filter,ModelFlux,fieldID,flux,fluxerr,deviations,zp,zpsys
pid,,,,,,,,,,
1000001,1,59580.033829,y,5.087947e-09,316,4.585038e-09,7.813779e-10,-0.643618,0.0,ab
14000001,1,59580.039637,y,5.088211e-09,1118,3.422685e-09,7.490886e-10,-2.223403,0.0,ab
13000001,1,59580.039190,y,5.088191e-09,1220,5.546394e-09,7.328528e-10,0.625231,0.0,ab
12000001,1,59580.038741,y,5.088171e-09,1212,3.953422e-09,7.083070e-10,-1.602058,0.0,ab
11000001,1,59580.038293,y,5.088150e-09,1110,4.288010e-09,7.245128e-10,-1.104383,0.0,ab
10000001,1,59580.037847,y,5.088130e-09,1021,5.127584e-09,7.563333e-10,0.052165,0.0,ab
9000001,1,59580.037401,y,5.088110e-09,957,4.508312e-09,7.839736e-10,-0.739563,0.0,ab
8000001,1,59580.036955,y,5.088089e-09,857,6.290365e-09,7.791733e-10,1.543015,0.0,ab
7000001,1,59580.036509,y,5.088069e-09,770,4.085100e-09,7.757777e-10,-1.292857,0.0,ab


In [23]:
bsim.write_lc(0, 'test_0.csv', 'csv', key=None)

0 a


In [25]:
dfcsv = pd.read_csv('test_0.csv')

In [25]:
bsim.write_simulation('test.hdf', 'hdf', key=None)

writing 0, False, None
0 a
writing 1, False, None
1 a


In [40]:
df = pd.read_hdf('test.hdf', key='0')

In [41]:
df.to_sql('')

,objid,expMJD,filter,ModelFlux,fieldID,flux,fluxerr,deviations,zp,zpsys
pid,,,,,,,,,,
1000000,0,59580.033829,y,0.000003,316,0.000003,1.640065e-09,0.066517,0.0,ab
14000000,0,59580.039637,y,0.000003,1118,0.000003,1.624962e-09,0.302472,0.0,ab
13000000,0,59580.039190,y,0.000003,1220,0.000003,1.617539e-09,-0.634322,0.0,ab
12000000,0,59580.038741,y,0.000003,1212,0.000003,1.606565e-09,-0.362741,0.0,ab
11000000,0,59580.038293,y,0.000003,1110,0.000003,1.613773e-09,-0.672460,0.0,ab
10000000,0,59580.037847,y,0.000003,1021,0.000003,1.628304e-09,-0.359553,0.0,ab
9000000,0,59580.037401,y,0.000003,957,0.000003,1.641323e-09,-0.813146,0.0,ab
8000000,0,59580.036955,y,0.000003,857,0.000003,1.639033e-09,-1.726283,0.0,ab
7000000,0,59580.036509,y,0.000003,770,0.000003,1.637419e-09,0.177426,0.0,ab


In [24]:
bsim.write_population('pop.hdf', method='hdf')

NameError: name 'bsim' is not defined

In [ ]:
sp.idxvalues

In [28]:
dfcsv.equals(df, )

False

In [24]:
bsim = BasicSimulation(population=sp, model=Model, pointings=)

SyntaxError: invalid syntax (<ipython-input-24-a94f7f6b7efb>, line 1)

In [21]:
bsim.write_lc(0, 'lc_0.hdf', 'hdf')

TypeError: to_hdf() takes exactly 3 arguments (2 given)

In [ ]:
pd.DataFrame.to_hdf()

In [28]:
bsim.write_simulation('test.hdf', 'hdf', key='test')

writing 0, False, test
0 a
writing 1, False, test
1 a


In [29]:
simdf = pd.read_hdf('test.hdf', key='test')

In [30]:
simdf

,objid,expMJD,filter,ModelFlux,fieldID,flux,fluxerr,deviations,zp,zpsys
pid,,,,,,,,,,
1000001,1,59580.033829,y,5.087947e-09,316,4.585038e-09,7.813779e-10,-0.643618,0.0,ab
14000001,1,59580.039637,y,5.088211e-09,1118,3.422685e-09,7.490886e-10,-2.223403,0.0,ab
13000001,1,59580.039190,y,5.088191e-09,1220,5.546394e-09,7.328528e-10,0.625231,0.0,ab
12000001,1,59580.038741,y,5.088171e-09,1212,3.953422e-09,7.083070e-10,-1.602058,0.0,ab
11000001,1,59580.038293,y,5.088150e-09,1110,4.288010e-09,7.245128e-10,-1.104383,0.0,ab
10000001,1,59580.037847,y,5.088130e-09,1021,5.127584e-09,7.563333e-10,0.052165,0.0,ab
9000001,1,59580.037401,y,5.088110e-09,957,4.508312e-09,7.839736e-10,-0.739563,0.0,ab
8000001,1,59580.036955,y,5.088089e-09,857,6.290365e-09,7.791733e-10,1.543015,0.0,ab
7000001,1,59580.036509,y,5.088069e-09,770,4.085100e-09,7.757777e-10,-1.292857,0.0,ab


In [ ]:
sp.positions

In [ ]:
model = Model()

In [ ]:
sp.modelparams(0)

In [ ]:
sp.modelparams(0)

In [ ]:
model.setModelParameters(**sp.modelparams(0))

In [ ]:
model.sn.SNstate

In [ ]:
sn = SNObject()

In [ ]:
params

In [ ]:
sn.setCoords(params['ra'], params['dec'])

In [ ]:
sn.set(**params)

In [ ]:
sn.mwEBVfromMaps()

In [ ]:
sn.SNstate

In [ ]:
model = Model()

In [ ]:
params = sp.modelparams(0).copy()

In [ ]:
for key in ('ra', 'dec'):
    params.pop(key)

In [ ]:
params

In [ ]:
sn.set(**params)

In [ ]:
sn.SNstate

In [ ]:
model.setModelParameters(**sp.modelparams(0))

In [ ]:
model.sn.SNstate

In [ ]:
bandpassobj = bandpassdict['y']

In [ ]:
model.setModelParameters(**sp.modelparams(0))

In [ ]:
model.modelFlux(59580., bandpassobj=bandpassobj)

In [ ]:
bsim = BasicSimulation(sp, model, pointings, rng=np.random.RandomState(0), 
                       maxObsHistID=1000000, pointingColumnDict=None,
                       timeRange=
                      pruneWithRadius=False)

In [ ]:
bsim.model.maxMjd

In [ ]:
bsim.model.r

In [ ]:
scrtch__

In [ ]:
opsout = _OpSimOutput.fromOpSimDB('/Users/rbiswas/data/LSST/OpSimData/minion_1016_sqlite.db',
                                 )

In [ ]:
pointings = opsout.summary.query('expMJD < 59580.04')

In [ ]:
pointings.expMJD.min()

In [ ]:
len(pointings)

In [ ]:
pointings.to_csv('check_poinx``tings.csv')

In [ ]:
!head check_pointings.csv

In [ ]:
from var

In [ ]:
from varsim import BaseExcep

In [ ]:
import pandas as pd

In [ ]:
import numpy as np

In [ ]:
df = pd.DataFrame()

In [ ]:
df['objid'] = np.arange(10)

In [ ]:
df

In [24]:
280 *2 /4 + 20

160

In [116]:
from collections import namedtuple, OrderedDict as odict

In [129]:
keys = 'abcde'
vals = np.arange(5)
idx = 'pq'
s1 = (dict( (k, v) for (i, k,v) in zip(idx, keys, vals)))
s2 = (dict( (k, v) for (i, k,v) in zip(idx, keys, vals)))



In [117]:
s = namedtuple(idx, s1)

In [123]:
s = list()
_ = list(s.append(x) for x in (s1, s2))

In [127]:
s1

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4}

In [102]:
xx = list()

In [107]:
xx = tuple(s)

In [103]:
xx.append(s)

In [133]:
idx = (l for l in idx)

In [135]:
df = pd.DataFrame(s, index=idx)

In [136]:
df

,a,b,c,d,e
p,0,1,2,3,4
q,0,1,2,3,4


In [57]:
df = pd.DataFrame()

In [58]:
type(s)

dict

In [56]:
df.append(s)

TypeError: Can only append a Series if ignore_index=True or if the Series has a name